In [ ]:
%matplotlib inline

In [ ]:
import os
import LFPy
import neuron
import numpy as np
import example_network_methods as methods
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
from matplotlib.gridspec import GridSpec
from plotting import annotate_subplot

In [ ]:
plt.rcParams.update({
    'axes.xmargin': 0.01,
    'axes.ymargin': 0.01,
    'font.size': 14,
    'legend.fontsize': 12,
    'axes.titlesize': 14,
})

In [ ]:
np.random.seed(1234)

In [ ]:
neuron.load_mechanisms('mod')

neuron.h.celsius = 34 # does nothing? celsius=34 seems hardcoded in Hay2011 NMODL files

In [ ]:
def get_polycol_from_cell(custom_fun, 
                          custom_fun_args,
                          morphology='BallAndSticks_E.hoc',
                          templatefile='BallAndSticksTemplate.hoc', 
                          templatename='BallAndSticksTemplate',
                          templateargs=None,
                          color='C0',
                          label='E', 
                          pos_args=dict(x=0, y=0, z=0),
                          rotation_args=dict(x=0, y=0, z=0)):
    cellParameters = dict(
        morphology=morphology,
        templatefile=templatefile,
        templatename=templatename,
        custom_fun=custom_fun,
        custom_fun_args=custom_fun_args,
        templateargs=templateargs,
        delete_sections=True,
    )
    cell = LFPy.TemplateCell(**cellParameters)    
    cell.set_pos(**pos_args)
    cell.set_rotation(**rotation_args)

    zips = []
    for x, z in cell.get_idx_polygons(projection=('x', 'z')):
        zips.append(list(zip(x, z)))
    polycol = PolyCollection(zips,
                             edgecolors=color,
                             facecolors=color,
                             linewidths=0.25,
                             label=label)
    return polycol

In [ ]:
def sim_response_ball_n_sticks(custom_fun, custom_fun_args,
                 Vrest=-65, amp=1.,
                 morphology='BallAndSticks_E.hoc', 
                 noise=None):
    cellParameters = dict(
        morphology=morphology,
        templatefile='BallAndSticksTemplate.hoc',
        templatename='BallAndSticksTemplate',
        custom_fun=custom_fun,
        custom_fun_args=custom_fun_args,  # [dict(Vrest=Vrest)],
        v_init=Vrest,
        dt=2**-4,
        tstop=1700,
        templateargs=None,
        delete_sections=True,
    )

    cell = LFPy.TemplateCell(**cellParameters)

    stim0 = LFPy.StimIntElectrode(cell=cell, idx=0, pptype='IClamp',
                                  record_current=True,
                                  amp=-amp, dur=100, delay=1100)
    stim1 = LFPy.StimIntElectrode(cell=cell, idx=0, pptype='IClamp',
                                  record_current=True,
                                  amp=amp, dur=100, delay=1300)

    stim2 = LFPy.StimIntElectrode(cell=cell, idx=0, pptype='IClamp',
                                  record_current=True,
                                  amp=0, dur=100, delay=1500)
    if noise is not None:
        noise.play(cell._hoc_stimlist[2]._ref_amp, cell.dt)
    
    cell.simulate()

    # show morphology
    if False:
        if custom_fun == [set_active_hay2011]:
            zips = []
            for x, z in cell.get_idx_polygons(projection=('x', 'z')):
                zips.append(list(zip(x, z)))
            polycol = PolyCollection(zips,
                                     edgecolors='none',
                                     facecolors='gray')

            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.add_collection(polycol)
            ax.axis(ax.axis('equal'))

    return cell.tvec, cell.somav, stim0.i + stim1.i + stim2.i

    del stim0, stim1, stim2, cell, morphology

In [ ]:
def sim_response_hay2011(templatefile, templatename, custom_fun, custom_fun_args,
                         Vrest=-65, amp=1., noise=None):
    cellParameters = dict(
        morphology='L5bPCmodelsEH/morphologies/cell1.asc',
        templatefile=templatefile,
        templatename=templatename,
        templateargs='L5bPCmodelsEH/morphologies/cell1.asc',
        passive=False,
        custom_fun=custom_fun,
        custom_fun_args=custom_fun_args,
        v_init=Vrest,
        dt=2**-4,
        tstop=1700,
        delete_sections=True,
    )

    cell = LFPy.TemplateCell(**cellParameters)
    cell.set_rotation(**dict(x=4.729, y=-3.166))

    stim0 = LFPy.StimIntElectrode(cell=cell, idx=0, pptype='IClamp',
                                  record_current=True,
                                  amp=-amp, dur=100, delay=1100)
    stim1 = LFPy.StimIntElectrode(cell=cell, idx=0, pptype='IClamp',
                                  record_current=True,
                                  amp=amp, dur=100, delay=1300)
    stim2 = LFPy.StimIntElectrode(cell=cell, idx=0, pptype='IClamp',
                                  record_current=True,
                                  amp=0, dur=100, delay=1500)
    if noise is not None:
        noise.play(cell._hoc_stimlist[2]._ref_amp, cell.dt)
    
    cell.simulate()

    return cell.tvec, cell.somav, stim0.i + stim1.i + stim2.i

    del stim0, stim1, stim2, cell, templatefile, custom_fun, custom_fun_args,

In [ ]:
# create some noisy stimuli trace
dt = 2**-4
delay = 1500
dur = 100
sigma = 0.0025
noise = np.r_[np.zeros(int(delay / dt)), (np.random.randn(int(dur / dt)) * sigma).cumsum()]
noise = neuron.h.Vector(noise)

In [ ]:
fig = plt.figure(figsize=(16, 10))
gs = GridSpec(4, 3, hspace=0.3)

# morphologies
ax0 = fig.add_subplot(gs[:, 0], aspect='equal')

# E cell
polycol = get_polycol_from_cell(
    custom_fun=[methods.set_active_hay2011],
    custom_fun_args=[dict(Vrest=-65)],
    morphology='BallAndSticks_E.hoc',
    templatefile='BallAndSticksTemplate.hoc', 
    templatename='BallAndSticksTemplate',
    color='C0',
    label='E',
    pos_args=dict(x=0, y=0, z=0))
ax0.add_collection(polycol)

# I cell
polycol = get_polycol_from_cell(
    custom_fun=[methods.set_active_hay2011],
    custom_fun_args=[dict(Vrest=-65)],
    morphology='BallAndSticks_I.hoc',
    templatefile='BallAndSticksTemplate.hoc', 
    templatename='BallAndSticksTemplate',
    color='C1',
    label='I',
    pos_args=dict(x=150, y=0, z=0))
ax0.add_collection(polycol)

# Hay cell
polycol = get_polycol_from_cell(
    morphology='L5bPCmodelsEH/morphologies/cell1.asc',
    templatefile=['L5bPCmodelsEH/models/L5PCbiophys3.hoc',
                  'L5bPCmodelsEH/models/L5PCtemplate.hoc'],
    templatename='L5PCtemplate',
    templateargs='L5bPCmodelsEH/morphologies/cell1.asc',
    custom_fun=None,
    custom_fun_args=None,
    color='C0',
    label=r'$\mathrm{E}_\mathrm{Hay2011}$',
    pos_args=dict(x=400, y=0, z=0),
    rotation_args=dict(x=4.729, y=-3.166, z=0))
ax0.add_collection(polycol)

ax0.axis(ax0.axis('equal'))

ax0.set_xlabel('$x$ (µm)', labelpad=0)
ax0.set_ylabel('$y$ (µm)', labelpad=0)
ax0.legend(loc=2)


# stimulus current
ax1 = fig.add_subplot(gs[0, 1:])

# Vm ball-n-sticks E population
ax2 = fig.add_subplot(gs[1, 1:], sharex=ax1)

m = 'BallAndSticks_E.hoc'
_, v0, _ = sim_response_ball_n_sticks([methods.set_active_hay2011], [dict(Vrest=-65)], -65, .0, m, noise)
Vrest = v0[-1]
for j, (custom_fun, label) in enumerate(zip(
        [methods.set_active_hay2011,
         methods.set_Ih_linearized_hay2011,
         methods.set_frozen_hay2011,
         methods.set_pas_hay2011],
        ['active',
         'biophys:lin',
         'biophys:frozen'])):
    if custom_fun not in [methods.set_active_hay2011]:
        t, v, i = sim_response_ball_n_sticks([custom_fun, methods.make_cell_uniform],
                                             [dict(Vrest=Vrest), dict(Vrest=Vrest)],
                                             Vrest, .05, m, noise)
    else:
        t, v, i = sim_response_ball_n_sticks([custom_fun],
                                             [dict(Vrest=Vrest)],
                                             Vrest, .05, m, noise)
    if j == 0:
        ax1.plot(t[t>=1075], i[t>=1075])        
    ax2.plot(t[t>=1075], v[t>=1075], label=label)

    ax2.legend(loc=2)
    ax1.set_ylabel(r'$I_\mathrm{stim}$ (nA)', labelpad=0)
    ax2.set_ylabel(r'$V_\mathrm{m}$ (mV)', labelpad=0)

plt.setp(ax1.get_xticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)

# Vm ball-n-sticks I population
ax3 = fig.add_subplot(gs[2, 1:], sharex=ax1)
m = 'BallAndSticks_I.hoc'
_, v0, _ = sim_response_ball_n_sticks([methods.set_active_hay2011], [dict(Vrest=-65)], -65, .0, m, noise)
Vrest = v0[-1]
for j, (custom_fun, label) in enumerate(zip(
        [methods.set_active_hay2011,
         methods.set_Ih_linearized_hay2011,
         methods.set_frozen_hay2011,
         methods.set_pas_hay2011],
        ['active',
         'lin',
         'frozen'])):
    if custom_fun not in [methods.set_active_hay2011]:
        t, v, i = sim_response_ball_n_sticks([custom_fun, methods.make_cell_uniform],
                                             [dict(Vrest=Vrest), dict(Vrest=Vrest)],
                                             Vrest, .05, m, noise)
    else:
        t, v, i = sim_response_ball_n_sticks([custom_fun],
                                             [dict(Vrest=Vrest)],
                                             Vrest, .05, m, noise)
    ax3.plot(t[t>=1075], v[t>=1075], label=label)
    ax3.set_ylabel(r'$V_\mathrm{m}$ (mV)', labelpad=0)

plt.setp(ax3.get_xticklabels(), visible=False)
    
# Vm Hay2011 E population
ax4 = fig.add_subplot(gs[3, 1:], sharex=ax1)

templatefiles = [
    ['L5bPCmodelsEH/models/L5PCbiophys3.hoc',
     'L5bPCmodelsEH/models/L5PCtemplate.hoc'],
    ['L5bPCmodelsEH/models/L5PCbiophys3_lin.hoc',
     'L5bPCmodelsEH/models/L5PCtemplate_lin.hoc'],
    ['L5bPCmodelsEH/models/L5PCbiophys3_frozen.hoc',
     'L5bPCmodelsEH/models/L5PCtemplate_frozen.hoc'],
]

templatenames = [
    'L5PCtemplate',
    'L5PCtemplate_lin',
    'L5PCtemplate_frozen',
    ]

_, v0, _ = sim_response_hay2011(
    ['L5bPCmodelsEH/models/L5PCbiophys3.hoc',
     'L5bPCmodelsEH/models/L5PCtemplate.hoc'],
    'L5PCtemplate',
     None, None, -77, .0)
Vrest = v0[-1]

custom_funs = [
    None,
    [methods.set_V_R,
     methods.make_cell_uniform],
    [methods.set_V_R,
     methods.make_cell_uniform],
]

custom_funs_args = [
    None,
    [dict(Vrest=Vrest)] * 2,
    [dict(Vrest=Vrest)] * 2,
    ]

for j, (templatefile, templatename, custom_fun, custom_fun_args, label) in enumerate(zip(
        templatefiles,
        templatenames,
        custom_funs,
        custom_funs_args,
        ['biophys3', 'lin', 'frozen']
        )):
    t, v, i = sim_response_hay2011(templatefile, templatename, custom_fun, custom_fun_args,
                                   Vrest, 0.05, noise)
    ax4.plot(t[t>=1075], v[t>=1075], label=label)

ax4.set_ylabel(r'$V_\mathrm{m}$ (mV)', labelpad=0)
ax4.set_xlabel('$t$ (ms)', labelpad=0)

for ax, title in zip([ax2, ax3, ax4], ['E', 'I', r'$\mathrm{E}_\mathrm{Hay2011}$']):
    ax.set_title(title)

for ax in [ax0, ax1, ax2, ax3, ax4]:
    lines=['right', 'top']
    for loc, spine in ax.spines.items():
        if loc in lines:
            spine.set_color('none')

# add panel labels
annotate_subplot(ax0, ncols=2, nrows=1, letter='A', linear_offset=0.02)
for ax, letter in zip([ax1, ax2, ax3, ax4], 'BCDE'): 
    annotate_subplot(ax, ncols=2, nrows=4, letter=letter, linear_offset=0.02)

if not os.path.isdir('figures'):
    os.mkdir('figures')
fig.savefig(os.path.join('figures', 'figure02.pdf'), bbox_inches='tight')